# Text Analysis on prince-toronto data

In [1]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\prince-toronto.csv', encoding = 'utf-8', header = None)
df.columns = ['src_tweet_id', 'src_user_id', 'src_tweet','src_date','reply_tweet_id','reply_user_id','reply_tweet','reply_date', 'label']
df.reply_tweet = df.reply_tweet.replace(np.nan,"")

In [3]:
src_tw_df = df.iloc[:,[0,1,2,3,8]].copy()
src_tw_df = src_tw_df.drop_duplicates()
src_tw_df

,src_tweet_id,src_user_id,src_tweet,src_date,label
0,529590949136326656,842564341,"As an aside, Toronto is home to @magpietoronto...",Tue Nov 04 11:08:16 +0000 2014,0
1,529695075560071168,134586361,Prince loves this city. He fell for the Toront...,Tue Nov 04 18:02:01 +0000 2014,0
3,529755373721632768,1395136699,PRINCE gives Toronto purple balls .....,Tue Nov 04 22:01:38 +0000 2014,0
4,529761572881985536,43451286,Dammit Prince. Toronto needs a win right now.,Tue Nov 04 22:26:16 +0000 2014,0
5,529399936434327552,23607802,can someone get us in 2 the secret prince show...,Mon Nov 03 22:29:15 +0000 2014,1
...,...,...,...,...,...
743,529778109290012673,102611790,this nigga Prince got all the unemployed middl...,Tue Nov 04 23:31:58 +0000 2014,1
744,529818929607475201,21137035,Live Nation wants to apologize for any inconve...,Wed Nov 05 02:14:11 +0000 2014,1
757,529828486975066112,20247647,#Toronto - sorry for the confusion today surro...,Wed Nov 05 02:52:09 +0000 2014,1
763,529830319982407680,526528241,From @DonnaGrantis on Facebook. @3RDEYEGIRL #T...,Wed Nov 05 02:59:26 +0000 2014,1


## Prepocessing
- Define function to process text at one time

In [4]:
def textprocessing(text):
    text = text.lower()

    text = " ".join([word for word in text.split() if 'http' not in word
                                and not word.startswith('@')])
                                #and word != 'RT'])
    import re
    def remove_punct(tweet):
        new_words = []
        for word in tweet:
            w = re.sub(r'[^\w\s]','',word) #remove everything except words and space
            w = re.sub(r'_','',w) #how to remove underscore as well
            new_words.append(w)

        return new_words
    text = "".join(remove_punct(text))
        
    from nltk.stem.porter import PorterStemmer
    stem = PorterStemmer()
    
    from nltk.tokenize import TweetTokenizer
    tknzr = TweetTokenizer(strip_handles=True)
    text = tknzr.tokenize(text)
    
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words("english"))
    def filterstopwords(tw):
        filter_stopwords = []
        for w in tw:
            if w not in stop_words:
                filter_stopwords.append(w)
        return filter_stopwords
    text = filterstopwords(text)
    text = " ".join(text)
    
    return text

In [5]:
print("original text: ", df.reply_tweet.iloc[20])
print("cleaned text: ", textprocessing(df.reply_tweet.iloc[20]))

original text:  @FOHPhoto @AUXTV I need to hear chocolate rain live....
cleaned text:  need hear chocolate rain live


### applied on original data (**df**)

In [6]:
df['cleaned_src_tw'] = df.src_tweet.apply(textprocessing)
df['cleaned_reply_tw'] = df.reply_tweet.apply(textprocessing)

In [7]:
df.head()

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label,cleaned_src_tw,cleaned_reply_tw
0,529590949136326656,842564341,"As an aside, Toronto is home to @magpietoronto...",Tue Nov 04 11:08:16 +0000 2014,5.295916e+17,1.538385e+09,"@PrinceMuseum @magpietoronto @3RDEYEGIRL Lol, ...",Tue Nov 04 11:11:03 +0000 2014,0,aside toronto home prince get lot stage clothes,lol hopefully stocking summer range 2 pop 2 au...
1,529695075560071168,134586361,Prince loves this city. He fell for the Toront...,Tue Nov 04 18:02:01 +0000 2014,5.296965e+17,6.211173e+07,“@ellhah: Prince loves this city. He fell for ...,Tue Nov 04 18:07:49 +0000 2014,0,prince loves city fell toronto woman trap happ...,ellhah prince loves city fell toronto woman tr...
2,529695075560071168,134586361,Prince loves this city. He fell for the Toront...,Tue Nov 04 18:02:01 +0000 2014,5.296980e+17,4.464473e+07,@ellhah Prince x Red Drink Boutique invite onl...,Tue Nov 04 18:13:30 +0000 2014,0,prince loves city fell toronto woman trap happ...,prince x red drink boutique invite party story...
3,529755373721632768,1395136699,PRINCE gives Toronto purple balls .....,Tue Nov 04 22:01:38 +0000 2014,NaN,NaN,,NaN,0,prince gives toronto purple balls,
4,529761572881985536,43451286,Dammit Prince. Toronto needs a win right now.,Tue Nov 04 22:26:16 +0000 2014,NaN,NaN,,NaN,0,dammit prince toronto needs win right,


### applied on source tweet data (**src_tw_df**) 

In [8]:
src_tw_df['cleaned_src_tw'] = src_tw_df.src_tweet.apply(textprocessing)

## Sentiment Analysis

In [9]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [10]:
def get_sentiment(tweet):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(tweet)
    if vs['compound'] >=0.05:
        sentiment_label = 'Positive'
    elif (vs['compound'] > -0.05) & (vs['compound'] < 0.05):
        sentiment_label = 'Neutral'
    elif vs['compound']<= -0.05:
        sentiment_label = 'Negative' 
    result = sentiment_label
    return result

### applied on original df

In [11]:
df['src_sentiment']= df.cleaned_src_tw.apply(get_sentiment)
df['reply_sentiment']= df.cleaned_reply_tw.apply(get_sentiment)

In [12]:
df.head(1)

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label,cleaned_src_tw,cleaned_reply_tw,src_sentiment,reply_sentiment
0,529590949136326656,842564341,"As an aside, Toronto is home to @magpietoronto...",Tue Nov 04 11:08:16 +0000 2014,5.295916e+17,1.538385e+09,"@PrinceMuseum @magpietoronto @3RDEYEGIRL Lol, ...",Tue Nov 04 11:11:03 +0000 2014,0,aside toronto home prince get lot stage clothes,lol hopefully stocking summer range 2 pop 2 au...,Neutral,Positive


### applied on source tweet data only 

In [13]:
### source tweet data only 
src_tw_df['src_sentiment']=  src_tw_df['cleaned_src_tw'].apply(get_sentiment)

### Visualization

In [ ]:
print(df[df.label ==1]['reply_sentiment'].value_counts())
plt.figure(figsize=(8,6))
sns.countplot(x='reply_sentiment',data= df[df['label'] == 1])

plt.suptitle('Sentiment analysis on Reply tweets under Catergory Rumour',fontsize=16)
plt.title('Reply tweets show more positive attitude towards rumour tweet',fontsize=12,color='grey')
plt.savefig('graph/prince-toronto/senti_reply_rumour.pdf',dpi=300)

In [ ]:
print(df[df.label ==0]['reply_sentiment'].value_counts())
plt.figure(figsize=(8,6))
sns.countplot(x='reply_sentiment',data= df[df['label'] == 0])

plt.suptitle('Sentiment analysis on Reply tweets under Catergory Non-Rumour',fontsize=16)
plt.title('Reply tweets show more neutrual attitude towards non-rumour tweet',fontsize=12,color='grey')
plt.savefig('graph/prince-toronto/senti_reply_nonrumour.pdf',dpi=300)

In [ ]:
print(src_tw_df['src_sentiment'].value_counts())
sns.countplot(x='src_sentiment',data =src_tw_df,hue='label')

plt.suptitle('Sentiment analysis on Source Tweets',fontsize=16)
plt.title('Rumour source tweets show more neutral attitude.',fontsize=12,color='grey')
plt.savefig('graph/prince-toronto/senti_source.pdf',dpi=300)

### save data to csv 

In [ ]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\prince-toronto-df.csv',index=False)
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\prince-toronto-src.csv',index=False)

## Keywords Extraction
### WordCloud

In [ ]:
from wordcloud import WordCloud

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(docx)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()

#### Reply Tweets

In [ ]:
#nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [ ]:
reply_sen = df.cleaned_reply_tw.apply(word_tokenize)
reply_tw_list = []
for sen in reply_sen:
    for token in sen:
        reply_tw_list.append(token)
reply_tw_doc = " ".join(reply_tw_list)

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(reply_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/prince-toronto/wordcloud_reply.pdf')

In [ ]:
# positive 
posi_reply_sen = df[df.reply_sentiment=='Positive'].cleaned_reply_tw.apply(word_tokenize)
posi_reply_tw_list = []
for sen in posi_reply_sen:
    for token in sen:
        posi_reply_tw_list.append(token)
posi_reply_tw_doc = " ".join(posi_reply_tw_list)


In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(posi_reply_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/prince-toronto/wordcloud_reply_posi.pdf')

#### Source Tweet

In [ ]:
src_sen = src_tw_df.cleaned_src_tw.apply(word_tokenize)
src_tw_list = []
for sen in src_sen:
    for token in sen:
        src_tw_list.append(token)

src_tw_doc = " ".join(src_tw_list)

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(src_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/prince-toronto/wordcloud_src.pdf')

### Common

In [ ]:
from collections import Counter

In [ ]:
def get_tokens(docx,num=30):
    word_tokens = Counter(docx)
    most_common = word_tokens.most_common(num)
    result = dict(most_common)
    return result

In [ ]:
pd.DataFrame(get_tokens(reply_tw_list).items(),columns=['word','freq']).plot(kind='bar',x='word',y='freq')
plt.title("Most frequent words among reply tweets are 'Prince','show','tonight'.")

In [ ]:
pd.DataFrame(get_tokens(src_tw_list).items(),columns=['word','freq']).plot(kind='bar',x='word',y='freq')
plt.title("Most frequent words among source tweets are 'Price', 'show', 'tonight'")


## Emotion Detection

### text2emotion package
https://snyk.io/advisor/python/text2emotion 

In [14]:
import text2emotion as te

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
emotion = df.cleaned_reply_tw.apply(te.get_emotion)

In [16]:
emo = emotion.apply(lambda x: max(x,key=x.get))
df['emotion']=emo

In [17]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\prince-toronto-df.csv',index=False)

In [18]:
emotion1 = src_tw_df.cleaned_src_tw.apply(te.get_emotion)
emo1 = emotion.apply(lambda x: max(x,key=x.get))
src_tw_df['emotion']=emo1

In [19]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\prince-toronto-src.csv',index=False)